In [1]:
!wget https://qnamakerstore.blob.core.windows.net/qnamakerdata/editorial/english/qna_chitchat_professional.tsv

--2024-07-28 07:09:02--  https://qnamakerstore.blob.core.windows.net/qnamakerdata/editorial/english/qna_chitchat_professional.tsv
Resolving qnamakerstore.blob.core.windows.net (qnamakerstore.blob.core.windows.net)... 20.60.81.165
Connecting to qnamakerstore.blob.core.windows.net (qnamakerstore.blob.core.windows.net)|20.60.81.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 981694 (959K) [text/plain]
Saving to: ‘qna_chitchat_professional.tsv’

qna_chitchat_profes 100%[===================>] 958.69K  --.-KB/s    in 0.1s    

2024-07-28 07:09:03 (7.11 MB/s) - ‘qna_chitchat_professional.tsv’ saved [981694/981694]



In [2]:
import pandas as pd
pd.read_csv("qna_chitchat_professional.tsv", sep="\t")[["Question", "Answer"]].to_csv("qna_chitchat_professional.tsv", index=False, sep="\t")

# 1. Tokenisation



In [3]:
pip install sentencepiece torch pandas


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
import pandas as pd
import sentencepiece as spm
import torch


data = pd.read_csv("qna_chitchat_professional.tsv", sep="\t")
questions = data['Question'].tolist()
answers = data['Answer'].tolist()

# Train SentencePiece model
spm.SentencePieceTrainer.Train(
    '--input=qna_chitchat_professional.tsv --model_prefix=tokenizer --vocab_size=1000 --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3 --user_defined_symbols=<sos>,<eos>'
)

# Load tokenizer model
sp = spm.SentencePieceProcessor()
sp.Load("tokenizer.model")


def encode_sentences(sentences):
    return [sp.encode_as_ids('<sos> ' + s + ' <eos>') for s in sentences]

# Encode questions and answers
encoded_questions = encode_sentences(questions)
encoded_answers = encode_sentences(answers)


print("Encoded sample question:", encoded_questions[0])
print("Encoded sample answer:", encoded_answers[0])


Encoded sample question: [15, 4, 42, 9, 65, 531, 14, 15, 5]
Encoded sample answer: [15, 4, 7, 20, 8, 12, 19, 16, 292, 6, 15, 5]


In [5]:
vocab_list = [(sp.id_to_piece(idx), idx) for idx in range(sp.GetPieceSize())]


print(vocab_list[:20])

[('<pad>', 0), ('<unk>', 1), ('<s>', 2), ('</s>', 3), ('<sos>', 4), ('<eos>', 5), ('.', 6), ('▁I', 7), ("'", 8), ('▁you', 9), ('s', 10), ('▁to', 11), ('t', 12), ('m', 13), ('?', 14), ('▁', 15), ('▁a', 16), (',', 17), ('▁me', 18), ('▁have', 19)]


# 2. Padding



In [6]:
import torch
from torch.nn.utils.rnn import pad_sequence


tensor_questions = [torch.tensor(seq) for seq in encoded_questions]
tensor_answers = [torch.tensor(seq) for seq in encoded_answers]


padded_questions = pad_sequence(tensor_questions, batch_first=True, padding_value=0)
padded_answers = pad_sequence(tensor_answers, batch_first=True, padding_value=0)


In [7]:
print("Padded Questions Shape:", padded_questions.shape)
print("Padded Answers Shape:", padded_answers.shape)
print("Padded Questions:", padded_questions)
print("Padded Answers:", padded_answers)


Padded Questions Shape: torch.Size([9793, 28])
Padded Answers Shape: torch.Size([9793, 48])
Padded Questions: tensor([[15,  4, 42,  ...,  0,  0,  0],
        [15,  4, 42,  ...,  0,  0,  0],
        [15,  4, 42,  ...,  0,  0,  0],
        ...,
        [15,  4,  7,  ...,  0,  0,  0],
        [15,  4, 15,  ...,  0,  0,  0],
        [15,  4,  7,  ...,  0,  0,  0]])
Padded Answers: tensor([[15,  4,  7,  ...,  0,  0,  0],
        [15,  4,  7,  ...,  0,  0,  0],
        [15,  4,  7,  ...,  0,  0,  0],
        ...,
        [15,  4,  7,  ...,  0,  0,  0],
        [15,  4,  7,  ...,  0,  0,  0],
        [15,  4,  7,  ...,  0,  0,  0]])


# 3. Split



In [8]:
pip install scikit-learn


In [9]:
from sklearn.model_selection import train_test_split

train_questions, test_questions, train_answers, test_answers = train_test_split(
    padded_questions, padded_answers, test_size=0.2, random_state=42
)

In [10]:
print("Train Questions Shape:", train_questions.shape)
print("Test Questions Shape:", test_questions.shape)
print("Train Answers Shape:", train_answers.shape)
print("Test Answers Shape:", test_answers.shape)


Train Questions Shape: torch.Size([7834, 28])
Test Questions Shape: torch.Size([1959, 28])
Train Answers Shape: torch.Size([7834, 48])
Test Answers Shape: torch.Size([1959, 48])


# 4. Simple LSTM



In [11]:

vocab_dict = {idx: sp.id_to_piece(idx) for idx in range(sp.GetPieceSize())}

def indices_to_words(indices):
    return [vocab_dict.get(idx, '') for idx in indices]


In [107]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.utils.rnn as rnn_utils
import random




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


class BidirectionalLSTMQAModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(BidirectionalLSTMQAModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.decoder = nn.LSTM(embedding_dim, hidden_dim * 2, batch_first=True)  # *2 because bidirectional
        self.fc = nn.Linear(hidden_dim * 2, vocab_size)

    def forward(self, src, src_lengths, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        max_len = trg.shape[1]
        vocab_size = self.fc.out_features

        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, max_len, vocab_size).to(src.device)

        # Embed the source sequence
        embedded_src = self.embedding(src)

        # Pack the embedded sequence
        packed_src = rnn_utils.pack_padded_sequence(embedded_src, src_lengths.cpu(), batch_first=True, enforce_sorted=False)

        # Encode the source sequence
        packed_outputs, (hidden, cell) = self.encoder(packed_src)

        # Unpack the sequence
        encoder_outputs, _ = rnn_utils.pad_packed_sequence(packed_outputs, batch_first=True)

        # Concatenate forward and backward hidden states
        hidden = torch.cat((hidden[0], hidden[1]), dim=1).unsqueeze(0)
        cell = torch.cat((cell[0], cell[1]), dim=1).unsqueeze(0)

        input = trg[:, 0]

        for t in range(1, max_len):
            # Embed the input
            embedded_input = self.embedding(input).unsqueeze(1)

            # Pass through the decoder
            output, (hidden, cell) = self.decoder(embedded_input, (hidden, cell))

            # Get the prediction
            prediction = self.fc(output.squeeze(1))

            # Store the prediction
            outputs[:, t] = prediction

            teacher_force = random.random() < teacher_forcing_ratio

            top1 = prediction.argmax(1)
            input = trg[:, t] if teacher_force else top1

        return outputs

# Function to get lengths of non-padded sequence
def get_lengths(tensor):
    return (tensor != 0).sum(dim=1)

# Set hyperparameters
vocab_size = 1000
embedding_dim = 256
hidden_dim = 512
batch_size = 32
num_epochs = 5
learning_rate = 0.01


train_questions = train_questions.to(device)
train_answers = train_answers.to(device)

# Create DataLoader
train_dataset = TensorDataset(train_questions, train_answers)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = BidirectionalLSTMQAModel(vocab_size, embedding_dim, hidden_dim).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_idx, (batch_questions, batch_answers) in enumerate(train_loader):
        optimizer.zero_grad()

        # Get lengths of non-padded sequence
        src_lengths = get_lengths(batch_questions)

        # Forward pass with teacher forcing
        output = model(batch_questions, src_lengths, batch_answers, teacher_forcing_ratio=0.5)


        output = output[:, 1:].reshape(-1, vocab_size)
        target = batch_answers[:, 1:].reshape(-1)

        # Calculate loss
        loss = criterion(output, target)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_idx % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")
# Save the trained model
torch.save(model.state_dict(), "bidirectional_lstm_qa_model_packed_sequence.pth")
print("Model saved successfully.")

Using device: cuda
Epoch 1/5, Batch 0/221, Loss: 6.9031
Epoch 1/5, Batch 10/221, Loss: 3.8519
Epoch 1/5, Batch 20/221, Loss: 3.2981
Epoch 1/5, Batch 30/221, Loss: 2.7097
Epoch 1/5, Batch 40/221, Loss: 2.1282
Epoch 1/5, Batch 50/221, Loss: 2.2021
Epoch 1/5, Batch 60/221, Loss: 1.2657
Epoch 1/5, Batch 70/221, Loss: 1.1626
Epoch 1/5, Batch 80/221, Loss: 1.2538
Epoch 1/5, Batch 90/221, Loss: 0.9466
Epoch 1/5, Batch 100/221, Loss: 1.2076
Epoch 1/5, Batch 110/221, Loss: 0.6332
Epoch 1/5, Batch 120/221, Loss: 0.8653
Epoch 1/5, Batch 130/221, Loss: 0.9481
Epoch 1/5, Batch 140/221, Loss: 1.2941
Epoch 1/5, Batch 150/221, Loss: 0.6580
Epoch 1/5, Batch 160/221, Loss: 0.6082
Epoch 1/5, Batch 170/221, Loss: 1.2794
Epoch 1/5, Batch 180/221, Loss: 0.7210
Epoch 1/5, Batch 190/221, Loss: 0.6184
Epoch 1/5, Batch 200/221, Loss: 0.5545
Epoch 1/5, Batch 210/221, Loss: 0.5873
Epoch 1/5, Batch 220/221, Loss: 0.4359
Epoch 1/5, Average Loss: 1.4733
Epoch 2/5, Batch 0/221, Loss: 1.6396
Epoch 2/5, Batch 10/221, L

In [109]:
from nltk.translate.bleu_score import corpus_bleu



test_questions = test_questions.to(device)
test_answers = test_answers.to(device)

# Create DataLoader for test data
test_dataset = TensorDataset(test_questions, test_answers)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

def evaluate_bleu(model, loader, vocab_dict):
    model.eval()
    references = []
    hypotheses = []

    with torch.no_grad():
        for batch_questions, batch_answers in loader:
            # Get lengths of non-padded sequence
            src_lengths = get_lengths(batch_questions)

            # Use batch_questions as both source and target for inference
            output = model(batch_questions, src_lengths, batch_questions, teacher_forcing_ratio=0)
            output = output.argmax(dim=2)

            for i in range(output.size(0)):
                hypothesis = indices_to_words(output[i].tolist())
                hypothesis = [word for word in hypothesis if word not in ['<pad>', '<eos>', '<unk>']]
                reference = indices_to_words(batch_answers[i].tolist())
                reference = [word for word in reference if word not in ['<pad>', '<eos>', '<unk>']]

                hypotheses.append(hypothesis)
                references.append([reference])

    # Calculate BLEU score
    bleu_score = corpus_bleu(references, hypotheses)
    return bleu_score

# Usage
test_bleu_score = evaluate_bleu(model, test_loader, vocab_dict)
print(f"Test BLEU Score: {test_bleu_score:.4f}")

Test BLEU Score: 0.5076


# 5. Attention Based LSTM



In [115]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttentionLSTMQAModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(AttentionLSTMQAModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)

        self.decoder = nn.LSTMCell(embedding_dim + hidden_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, vocab_size)

        # Adjust dimensions for key, value, and query
        self.key_layer = nn.Linear(hidden_dim * 2, hidden_dim)
        self.value_layer = nn.Linear(hidden_dim * 2, hidden_dim)
        self.query_layer = nn.Linear(hidden_dim, hidden_dim)

        self.attention = nn.Linear(hidden_dim, 1)

    def forward(self, src, trg):
        batch_size = src.shape[0]
        max_len = trg.shape[1]
        vocab_size = self.fc.out_features

        embedded_src = self.embedding(src)
        encoder_outputs, (hidden, cell) = self.encoder(embedded_src)

        # Combine bidirectional states
        hidden = hidden.sum(dim=0)
        cell = cell.sum(dim=0)

        keys = self.key_layer(encoder_outputs)
        values = self.value_layer(encoder_outputs)

        outputs = torch.zeros(batch_size, max_len, vocab_size).to(src.device)
        input = trg[:, 0]

        for t in range(1, max_len):
            query = self.query_layer(hidden)

            attention_weights = self.calculate_attention(query, keys)
            context = torch.bmm(attention_weights.unsqueeze(1), values).squeeze(1)

            rnn_input = torch.cat((self.embedding(input), context), dim=1)

            hidden, cell = self.decoder(rnn_input, (hidden, cell))
            output = self.fc(hidden)
            outputs[:, t] = output

            input = output.argmax(1)

        return outputs

    def calculate_attention(self, query, keys):
        energy = self.attention(torch.tanh(query.unsqueeze(1) + keys)).squeeze(2)
        return F.softmax(energy, dim=1)

In [116]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, TensorDataset
import os
import torch.optim as optim
import torch.nn.utils.rnn as rnn_utils
import random
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smoother = SmoothingFunction().method1

# Hyperparameters
vocab_size = 1000
embedding_dim = 512
hidden_dim = 1024
batch_size = 64
num_epochs = 20
learning_rate = 0.01

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_questions = train_questions.to(device)
train_answers = train_answers.to(device)
test_questions = test_questions.to(device)
test_answers = test_answers.to(device)

# Create DataLoaders
train_dataset = TensorDataset(train_questions, train_answers)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(test_questions, test_answers)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize the model, loss function, and optimizer
model = AttentionLSTMQAModel(vocab_size, embedding_dim, hidden_dim).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)

def indices_to_words(indices, vocab_dict):
    return [vocab_dict.get(idx, "<unk>") for idx in indices]

def evaluate_bleu(model, loader, vocab_dict):
    model.eval()
    references = []
    hypotheses = []

    with torch.no_grad():
        for batch_questions, batch_answers in loader:
            batch_questions = batch_questions.to(device)
            batch_answers = batch_answers.to(device)

            output = model(batch_questions, batch_answers)
            output = output.argmax(dim=2)  # Convert model outputs to token indices

            for i in range(output.size(0)):
                hypothesis = indices_to_words(output[i].tolist(), vocab_dict)
                hypothesis = [word for word in hypothesis if word not in ['<pad>', '<eos>', '<unk>']]
                reference = indices_to_words(batch_answers[i].tolist(), vocab_dict)
                reference = [word for word in reference if word not in ['<pad>', '<eos>', '<unk>']]

                hypotheses.append(hypothesis)
                references.append([reference])

    # Calculate BLEU score

    bleu_score = corpus_bleu(references,hypotheses, smoothing_function=smoother)
    return bleu_score

# Training loop
best_bleu_score = 0
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_questions, batch_answers in train_loader:
        optimizer.zero_grad()

        output = model(batch_questions, batch_answers)

        output = output[:, 1:].reshape(-1, vocab_size)
        target = batch_answers[:, 1:].reshape(-1)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # Evaluate BLEU score
    bleu_score = evaluate_bleu(model, test_loader, vocab_dict)

    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}, BLEU Score: {bleu_score:.4f}")

    # Save the best model
    if bleu_score > best_bleu_score:
        best_bleu_score = bleu_score
        torch.save(model.state_dict(), "best_attention_lstm_qa_model.pth")
        print(f"New best model saved with BLEU score: {best_bleu_score:.4f}")

print("Training completed.")
print(f"Best BLEU Score: {best_bleu_score:.4f}")

Epoch 1/20, Average Loss: 3.6485, BLEU Score: 0.0030
New best model saved with BLEU score: 0.0030
Epoch 2/20, Average Loss: 3.0224, BLEU Score: 0.0175
New best model saved with BLEU score: 0.0175
Epoch 3/20, Average Loss: 2.6311, BLEU Score: 0.0291
New best model saved with BLEU score: 0.0291
Epoch 4/20, Average Loss: 2.3012, BLEU Score: 0.0489
New best model saved with BLEU score: 0.0489
Epoch 5/20, Average Loss: 2.0573, BLEU Score: 0.0566
New best model saved with BLEU score: 0.0566
Epoch 6/20, Average Loss: 1.8723, BLEU Score: 0.0959
New best model saved with BLEU score: 0.0959
Epoch 7/20, Average Loss: 1.6777, BLEU Score: 0.1206
New best model saved with BLEU score: 0.1206
Epoch 8/20, Average Loss: 1.5316, BLEU Score: 0.1799
New best model saved with BLEU score: 0.1799
Epoch 9/20, Average Loss: 1.4130, BLEU Score: 0.2081
New best model saved with BLEU score: 0.2081
Epoch 10/20, Average Loss: 1.3194, BLEU Score: 0.2649
New best model saved with BLEU score: 0.2649
Epoch 11/20, Averag

In [124]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smoother = SmoothingFunction().method1

def generate_answer(model, question, vocab_dict):
    model.eval()
    with torch.no_grad():
        question = question.unsqueeze(0).to(device)
        max_length = 50

        sos_index = next(i for i, word in vocab_dict.items() if word == "<sos>")
        target = torch.tensor([[sos_index]]).to(device)

        for _ in range(max_length):
            output = model(question, target)
            next_token = output.argmax(dim=2)[:, -1]
            target = torch.cat([target, next_token.unsqueeze(1)], dim=1)

            eos_index = next(i for i, word in vocab_dict.items() if word == "<eos>")
            if next_token.item() == eos_index:
                break

    return target.squeeze().tolist()

# Load the best model
model.load_state_dict(torch.load("best_attention_lstm_qa_model.pth"))
model.eval()


sample_indices = random.sample(range(len(test_dataset)), 5)


individual_bleu_scores = []

for i, sample_index in enumerate(sample_indices, 1):
    question, true_answer = test_dataset[sample_index]

    # Generate predicted answer
    predicted_answer_indices = generate_answer(model, question, vocab_dict)


    question_words = [vocab_dict.get(idx.item(), "<unk>") for idx in question if idx.item() != 0]
    true_answer_words = [vocab_dict.get(idx.item(), "<unk>") for idx in true_answer if idx.item() != 0]
    predicted_answer_words = [vocab_dict.get(idx, "<unk>") for idx in predicted_answer_indices if idx != 0]

    # Remove special tokens
    special_tokens = {'<pad>', '<sos>', '<eos>', '<unk>'}
    question_words = [word for word in question_words if word not in special_tokens]
    true_answer_words = [word for word in true_answer_words if word not in special_tokens]
    predicted_answer_words = [word for word in predicted_answer_words if word not in special_tokens]

    print(f"\nExample {i}:")
    print(f"Question: {' '.join(question_words)}")
    print(f"True Answer: {' '.join(true_answer_words)}")
    print(f"Predicted Answer: {' '.join(predicted_answer_words)}")


    bleu_score = sentence_bleu([true_answer_words], predicted_answer_words, smoothing_function=smoother)
    individual_bleu_scores.append(bleu_score)
    print(f"BLEU Score: {bleu_score:.4f}")




Example 1:
Question: ▁ ▁S hould ▁I ▁call ▁my ▁dad ? ▁
True Answer: ▁ ▁I ▁wouldn ’ t ▁know ▁how ▁to ▁ad vise ▁about ▁this . ▁
Predicted Answer: ▁I ▁wouldn ’ t ▁know ▁how ▁to ▁ad vise ▁about ▁this . ▁
BLEU Score: 0.9260

Example 2:
Question: ▁ ▁Do ▁I ▁look ▁attractive ▁to ▁you ▁today ? ▁
True Answer: ▁ ▁H onestly , ▁I ▁can ' t ▁tell ▁one ▁way ▁or ▁the ▁other . ▁
Predicted Answer: ▁H onestly , ▁I ▁can ' t ▁tell ▁one ▁way ▁or ▁the ▁other . ▁
BLEU Score: 0.9355

Example 3:
Question: ▁ ▁How ' s ▁your ▁S unday ▁
True Answer: ▁ ▁Good , ▁thanks . ▁
Predicted Answer: ▁Good , ▁thanks . ▁
BLEU Score: 0.8187

Example 4:
Question: ▁ ▁Well ▁are n ' t ▁you ▁feeling ▁happy ? ▁
True Answer: ▁ ▁I ' m ▁quit e ▁happy , ▁thank ▁you . ▁
Predicted Answer: ▁I ' m ▁quit e ▁happy , ▁thank ▁you . ▁
BLEU Score: 0.9131

Example 5:
Question: ▁ ▁What ▁team ▁mad e ▁you ? ▁
True Answer: ▁ ▁People ▁created ▁me . ▁
Predicted Answer: ▁People ▁created ▁me . ▁
BLEU Score: 0.8187
